# Evaluate GPT-3 for Korean sentence completions

Inputs used here are formatted as one of the following.
- Nom-Nom: Subject1-Nom Object1-Acc V. Subject2-Nom Object1-Acc
- Nun-Nun: Subject1-Nun Object1-Acc V. Subject2-Nun Object1-Acc
- NC-Nom: Subject2-Nom Object1-Acc
- NC-Nun: Subject2-Nun Object1-Acc

Model inserts the continuation of the second partial sentence. Probability scores for the completion is calculated. 

In the Nun-Nun condition, it is pragmatically felicitious to continue the sentence with the contrasted predicate from the previous sentence.

In [1]:
# For generation
import os
import openai

import pandas as pd
from tqdm import tqdm 

In [2]:
# For analysis
import numpy as np
import pickle
import csv
import math
import ast
import re

In [3]:
# For tokenizing Korean
from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()

In [4]:
# Supply your own OpenAI API keys
keys = open('../openai-keys.txt')
lines = keys.readlines()
openai.organization = lines[0].rstrip()
openai.api_key = lines[1].rstrip()

In [5]:
def insert_gpt3(item):
    """
    Generate sentence continuation with GPT-3. 
    Print out the output in a dictionary format.
    """
    
    output = openai.Completion.create(
        model = "text-davinci-003",
        prompt = item,
        suffix = ".",
        max_tokens = 30, 
        temperature = 0, # argmax
        n = 1,
        logprobs = 1,
        stop = ["."] 
        # Although "\n" can indicate a phrase ending, having this resulted in empty responses for several prompts.
        # Having a new line after the prompt is not technically grammatical, but it is still considered in analysis.
        )
    
    output_dict = output.to_dict()['choices'][0].to_dict()
    
    return output_dict

In [6]:
def inserted_prob_gpt3(experiment_item):
    """
    Generate sentence continuation with GPT-3. 
    Print out the log probability for the inserted tokens in a dictionary format.
    """
    
    output = openai.Completion.create(
        model = "text-davinci-003",
        prompt = experiment_item,
        suffix = ".",
        max_tokens = 30,
        temperature = 0,
        n = 1,
        #stream = False,
        logprobs = 1,
        stop = ["."]
        )
    
    toplogprob = output.to_dict()['choices'][0].to_dict()['logprobs'].to_dict()["top_logprobs"]
    # We are using the ["top_logprobs"] because it notes each output token and its logprob together in a JSON entry.
    # Each ["top_logprobs"] is converted into a Python dictionary: output token as a key and logprob as a value. 
    
    response = []
    
    for i in range(0,len(toplogprob)):
        response.append(toplogprob[i].to_dict())
        
    return response

POS tag reference: http://kkma.snu.ac.kr/documents/index.jsp?doc=postag

In [7]:
def eval_relation(token_pos, cverb_pos):
    """
    Determine whether the model generated the same, 
    negated or different verb than the context verb. 
    
    'contrasted': context verb root with negation
    'repeated': context verb root
    'different': no context verb root
    """
    
    relation = ''
    negation = [('안','MAG'), ('못', 'MAG'), ('않', 'VXV'), ('못하', 'VX')]
    
    if (cverb_pos[0] in token_pos) and any(neg in negation for neg in token_pos):
        relation = 'contrasted'
        
    elif (cverb_pos[0] in token_pos) and not any(neg in negation for neg in token_pos):
        relation = 'repeated'
        
    else: relation = 'different' #includes result having no verb root
        
    return relation

In [8]:
negation = [('안', 'MAG'), ('못', 'MAG'), ('않', 'VXV'),('못하', 'VX')]
item = [('안', 'MAG'),('가', 'VV'), ('았', 'EPT'), ('습니다', 'EFN')]
root = [('받', 'VV')]

In [9]:
eval_relation(item,root)

'different'

In [10]:
def phrase_prob(item):
        logprob = []
        i = 0
        
        while i < len(item):
            if r'.' and r',' not in item[i].keys():
                logprob.append(*item[i].values())
                i += 1
            else:
                break
            
        return np.exp(sum(logprob))

In [12]:
def phrase_ended(item):
    i = 0 
    ended = str()
    enders = {'ECE', 'ECS', 'EFN', 'EFQ', 'EFO', 'EFA', 'EFI', 'EFR'}
    
    while i < len(item):
        if item[i][1] in enders:
            ended = 'ended'
            break
        else:
            i += 1

    if len(ended) == 0:
        if item[-1][1] not in enders:
            ended = 'not ended'
        
    return ended

In [14]:
def main_gpt3_df(filename):
    """
    Sentence continuations for each prompt given in a CSV format using GPT-3.
    Compile the output and POS tags of the created tokens into a dataframe.
    """
    
    data_path = "input/{FILE}.csv".format(FILE=filename)
    df_output = pd.read_csv(data_path)
    
    token = []
    tpos = []
    cpos = []
    rel = []
    phraseprob = []
    phraseend = []
    
    with tqdm(total=df_output.shape[0]) as pbar:
        for index, row in df_output.iterrows():
        
            prompt = row['full_prompt']
            
            t = insert_gpt3(prompt)
            token.append(t['text'])
            
            token_pos = kkma.pos(t['text']) #Korean pos tagger 
            tpos.append(token_pos)
            
            cverb_pos = kkma.pos(row['context_verb'])
            cpos.append(cverb_pos)
            
            relation = eval_relation(token_pos, cverb_pos)
            rel.append(relation)
            
            phrase_p = phrase_prob(inserted_prob_gpt3(prompt))
            phraseprob.append(phrase_p)
            
            phrase_end = phrase_ended(kkma.pos(t['text']))
            phraseend.append(phrase_end)
            
            pbar.update(1)
            
    df_output['token'] = token
    df_output['token_pos'] = tpos
    df_output['cverb_pos'] = cpos
    df_output['relation'] = rel
    df_output['phrase_prob'] = phraseprob
    df_output['phrase_ended'] = phraseend

    df_output.to_csv("{TASK}_gpt3_df_output.csv".format(TASK=filename), index=False)
    
    return df_output

In [15]:
main_gpt3_df('subj-insert-sample')

100%|█████████████████████████████████████████████| 4/4 [00:08<00:00,  2.17s/it]


,item_id,scenario_id,condition,scenario,target,full_prompt,context_verb,token,token_pos,cverb_pos,relation,phrase_prob,phrase_ended
0,1,1,nc-nom,지은이 친구들 다섯 명과 함께 마라톤 경주에 참여했습니다. 지은은 다른 약속이 있어...,민찬이가 메달을,민찬이가 메달을,받다,받았습니다,"[(받, VV), (았, EPT), (습니다, EFN)]","[(받, VV), (다, EFN)]",repeated,0.113311,ended
1,2,1,nc-nun,지은이 친구들 다섯 명과 함께 마라톤 경주에 참여했습니다. 지은은 다른 약속이 있어...,민찬이는 메달을,민찬이는 메달을,받다,받았습니다,"[(받, VV), (았, EPT), (습니다, EFN)]","[(받, VV), (다, EFN)]",repeated,0.060011,ended
2,3,1,nom-nom,지은이 친구들 다섯 명과 함께 마라톤 경주에 참여했습니다. 지은은 다른 약속이 있어...,보미가 메달을 받았어. 민찬이가 메달을,보미가 메달을 받았어. 민찬이가 메달을,받다,받았어,"[(받, VV), (았, EPT), (어, EFN)]","[(받, VV), (다, EFN)]",repeated,0.832638,ended
3,4,1,nun-nun,지은이 친구들 다섯 명과 함께 마라톤 경주에 참여했습니다. 지은은 다른 약속이 있어...,보미는 메달을 받았어. 민찬이는 메달을,보미는 메달을 받았어. 민찬이는 메달을,받다,받지 못했어,"[(받, VV), (지, ECD), (못하, VX), (었, EPT), (어, EFN)]","[(받, VV), (다, EFN)]",contrasted,0.392894,ended


In [16]:
#output = main_gpt3_df('subj-insert-full')

100%|█████████████████████████████████████████| 112/112 [03:43<00:00,  2.00s/it]


In [17]:
#pd.set_option('display.max_rows', 500)
#output

,item_id,scenario_id,condition,scenario,target,full_prompt,context_verb,token,token_pos,cverb_pos,relation,phrase_prob,phrase_ended
0,1,1,nc-nom,지은이 친구들 다섯 명과 함께 마라톤 경주에 참여했습니다. 지은은 다른 약속이 있어...,민찬이가 메달을,지은이 친구들 다섯 명과 함께 마라톤 경주에 참여했습니다. 지은은 다른 약속이 있어...,받다,"얻었고, 영훈이는 두 번째","[(얻, VV), (었, EPT), (고, ECE), (,, SP), (영, NNG...","[(받, VV), (다, EFN)]",different,0.195460,ended
1,2,2,nc-nom,영준과 은지는 다른 세 명의 친구들과 함께 오후 5시에 만나기로 약속했습니다. 영준...,채영이가 집을,영준과 은지는 다른 세 명의 친구들과 함께 오후 5시에 만나기로 약속했습니다. 영준...,나오다,"빠져서 늦게 왔고, 소연이","[(빠지, VV), (어서, ECD), (늦, VA), (게, ECD), (오, V...","[(나오, VV), (다, EFN)]",different,0.003906,ended
2,3,3,nc-nom,혜미와 혜미의 친구들이 연말 파티를 하기로 했습니다. 몇몇 친구들은 음식을 준비하고...,동준이가 음식을,혜미와 혜미의 친구들이 연말 파티를 하기로 했습니다. 몇몇 친구들은 음식을 준비하고...,만들다,"준비했고, 지연이가 파","[(준비, NNG), (하, XSV), (었, EPT), (고, ECE), (,, ...","[(만들, VV), (다, EFN)]",different,0.681167,ended
3,4,4,nc-nom,각 지역 신문은 매일 일기 예보 기사를 전합니다. 민준이 아현에게 오늘의 일기 예보...,부산 신문이 비 소식을,각 지역 신문은 매일 일기 예보 기사를 전합니다. 민준이 아현에게 오늘의 일기 예보...,알리다,전하고 있습니다,"[(전하, VV), (고, ECE), (있, VXV), (습니다, EFN)]","[(알리, VV), (다, EFN)]",different,0.108936,ended
4,5,5,nc-nom,홍보팀장이 다섯 명의 팀원들로부터 전달받은 프로젝트 보고서를 연구팀에 보냈습니다. ...,박상엽 사원이 계획서를,홍보팀장이 다섯 명의 팀원들로부터 전달받은 프로젝트 보고서를 연구팀에 보냈습니다. ...,쓰다,"작성하고, 이승호 사원이","[(작성, NNG), (하, XSV), (고, ECE), (,, SP), (이승, ...","[(쓰, VV), (다, EFN)]",different,0.380932,ended
5,6,6,nc-nom,승우가 친구들과 함께 온라인 추리 게임을 했습니다. 승우는 범인이 누구라고 생각하는...,범수가 단서를,승우가 친구들과 함께 온라인 추리 게임을 했습니다. 승우는 범인이 누구라고 생각하는...,알다,모두 찾아냈습니다,"[(모두, MAG), (찾아내, VV), (었, EPT), (습니다, EFN)]","[(알, VV), (다, EFN)]",different,0.000779,ended
6,7,7,nc-nom,민아와 상현이가 유기견을 돕는 봉사활동을 하고 있습니다. 민아는 교실 앞에 기부함을...,태성이가 먹을 것을,민아와 상현이가 유기견을 돕는 봉사활동을 하고 있습니다. 민아는 교실 앞에 기부함을...,주다,"기부했고, 지연이가 놀이","[(기부, NNG), (하, XSV), (었, EPT), (고, ECE), (,, ...","[(주, VXV), (다, EFN)]",different,0.623868,ended
7,8,8,nc-nom,상훈이 친구들과 함께 피구게임을 했고 혜미는 게임을 하지 않았습니다. 상훈은 게임 ...,건후가 공을,상훈이 친구들과 함께 피구게임을 했고 혜미는 게임을 하지 않았습니다. 상훈은 게임 ...,맞다,"\n\n""건후가 공을 치고 이겼","[("", SS), (건, NNM), (후, NNG), (가, JKS), (공, NN...","[(맞닿, VV)]",different,0.004099,ended
8,9,9,nc-nom,주희가 여러 명의 미술 작가들이 참여한 전시회에 구경을 갔지만 마감 시간이 되어서 ...,이강호 작가가 관심을,주희가 여러 명의 미술 작가들이 참여한 전시회에 구경을 갔지만 마감 시간이 되어서 ...,모으다,끌었어,"[(끌, VV), (었, EPT), (어, EFN)]","[(모으, VV), (다, EFN)]",different,0.025594,ended
9,10,10,nc-nom,윤지가 친구들의 연극 공연을 보러 갔습니다. 윤지는 공연에 늦게나마 도착했지만 도현...,은영이가 인기를,윤지가 친구들의 연극 공연을 보러 갔습니다. 윤지는 공연에 늦게나마 도착했지만 도현...,끌다,많이 받았어,"[(많이, MAG), (받, VV), (았, EPT), (어, EFN)]","[(끌, VV), (다, EFN)]",different,0.016221,ended


## To do/Notes

0. Anomaly in POS tagger
- row105 [(잡다, XR)] in cverb_pos, [(잡, VV)] in token_pos

1. Phrase ending vs. Sentence ending: what is more important for evaluation?
- We need to be able to know whether the non-contrastive relation is from tokens not having the verb or the generated verb token is actually non-contrastive

- Add eval_VP to check whether the output is a grammatical VP.


2. We can increase the max length enough that sentence endings are likely to appear, but this is an additional constraint we will feed in that we don't have to in human cloze tasks (Humans respond with one or two words without an additional instruction).
- Reflecting this, in stat analysis, we can only consider cases where phrase/sentence ending was present. If the sentence is not ended, GPT-3 results are not like human's cloze test, where people are likely to have the simpliest/shortest VP to end the sentence.